In [1]:
from sklearn.feature_selection import SelectFdr, f_classif
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import wilcoxon
from scipy.stats import ranksums
from statsmodels.stats.multitest import fdrcorrection
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif,f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, mean_squared_error, precision_score, jaccard_score, fowlkes_mallows_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from boruta import BorutaPy
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold

In [5]:
rna = pd.read_csv('D:/multiomics/2025june/rna_rb.csv', sep=',')
lipid = pd.read_csv('D:/multiomics/2025june/lipid_rb.csv', sep=',')
meta = pd.read_csv('D:/multiomics/2025june/meta_rb.csv', sep=',')

In [8]:
sets_to_intersect = [
    set(rna['id']),
    set(lipid['id']),
    set(meta['id'])
]

common_ids = set.intersection(*sets_to_intersect)

sorted_common_ids = sorted(common_ids)

meta = meta[meta['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
lipid = lipid[lipid['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
rna = rna[rna['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()

In [9]:
sum(rna['id']==meta['id']),sum(rna['id']==lipid['id'])

(188, 188)

In [10]:
def get_quantile(X_train, column_name,new_column_name,a,b,position):

    percentile_a = X_train[column_name].quantile(a)
    percentile_b = X_train[column_name].quantile(b)

    X_train[new_column_name] = 0
    X_train.loc[X_train[column_name] >= percentile_a, new_column_name] = 1

    column_to_move = new_column_name
    target_column = position

    column = X_train.pop(column_to_move)
    X_train.insert(target_column, column_to_move, column)
    
    return X_train

In [11]:
rna = get_quantile(rna, 'sVNT_M1_WT_RBD_10_times_dilution','sVNT_M1_WT_RBD',0.75,0,10)
meta = meta.reset_index(drop=True)
lipid = lipid.reset_index(drop=True)
rna = rna.reset_index(drop=True)

In [12]:
f_rna = rna.iloc[:,12:]
f_lipid = lipid.iloc[:,6:]
f_meta = meta.iloc[:,6:]
f_agender = rna.iloc[:,1:3]
label = rna.iloc[:,10]

In [13]:
cross_table = pd.crosstab(rna['sVNT_M1_WT_RBD'], rna['Batch'])

In [23]:
cross_table

Batch,Batch01,Batch02
sVNT_M1_WT_RBD,,
0,81,60
1,10,37


In [24]:
combine = pd.concat([f_rna, f_lipid, f_meta], axis=1) 
combine = pd.concat([f_agender, combine], axis=1)

In [25]:
bl = (
    rna['Batch'].astype(str).str.cat(rna['sVNT_M1_WT_RBD'].astype(str), sep='_')
)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combine, label, test_size=0.2,stratify=bl,random_state=42)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,bl.shape

((150, 44946), (38, 44946), (150,), (38,), (188,))

In [28]:
bl[X_train.index]

184    Batch02_0
46     Batch02_1
39     Batch02_1
94     Batch01_0
70     Batch02_0
         ...    
17     Batch02_1
29     Batch02_1
139    Batch01_0
126    Batch01_0
186    Batch01_0
Name: Batch, Length: 150, dtype: object

In [35]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from functools import partial
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import f_classif
from joblib import Parallel, delayed
import pandas as pd

def process_fold(train_idx, val_idx, i, X_train, y_train):
    print(f"Processing fold {i}")
    
    X_fold_train, y_fold_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
    
    F_values, p_values = f_classif(X_fold_train, y_fold_train)
    significant_features_mask = p_values < 0.05
    significant_features = X_fold_train.columns[significant_features_mask]
    print(f"Fold {i+1}: {len(significant_features)} significant features")
    X_fold_train_sig = X_fold_train[significant_features]

    estimator = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    boruta_selector = BorutaPy(
        estimator=estimator,
        n_estimators='auto', 
        verbose=0,          
        random_state=42,
        max_iter=100,     
        perc=95,alpha=0.05             
    )
    
    boruta_selector.fit(X_fold_train_sig.values, y_fold_train.values)
    

    selected_features_mask = boruta_selector.support_
    selected_features = significant_features[selected_features_mask]
    

    estimator.fit(X_fold_train_sig[selected_features], y_fold_train)
    importances = estimator.feature_importances_
    

    return [
        {
            'fold': i,
            'feature': feature,
            'importance': importance,
        }
        for feature, importance in zip(selected_features, importances)
    ]


def main():

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(partial(process_fold, X_train=X_train, y_train=y_train))(train_idx, val_idx, i)
        for i, (train_idx, val_idx) in enumerate(cv.split(X_train, bl[X_train.index]))
    )
    

    results = [item for sublist in results for item in sublist]
    

    results_df = pd.DataFrame(results)
    

    results_df = results_df.sort_values(['fold', 'importance'], ascending=[True, False])
    

    print(results_df.head())
    return results_df

if __name__ == '__main__':
    all_feature_counts = main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
C:\Users\win11\.conda\envs\pytorch\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.7min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.7min remaining:   44.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.8min finished


     fold                                    feature  importance
35      0   ENSG00000140463.14__BBS4__protein.coding    0.021758
110     0        ENSG00000280411.1__IGHV1.69D__other    0.021257
36      0  ENSG00000140829.12__DHX38__protein.coding    0.017193
131     0    HMDB0059701__1.6.7.Trimethylnaphthalene    0.015535
118     0                        LPC.17.1..rep..rep.    0.014766


In [36]:
feature_counts = all_feature_counts['feature'].value_counts()
common_features = feature_counts[feature_counts >= 8].index
df_filtered = all_feature_counts[all_feature_counts['feature'].isin(common_features)]

stats = df_filtered.groupby('feature')['importance'].agg(['sum', 'mean', 'count'])
stats = stats.sort_values('count', ascending=False)

top_feature = stats.index[:3]
feature_selected = list(top_feature)

In [37]:
feature_selected

['C09694__Juvenile.hormone.III',
 'C12859__Estradiol.valerate',
 'ENSG00000173862.3__FLJ20712__ncRNA']

In [51]:
# Prepare training and test sets
Xtrain = X_train[['Age', 'Gender'] + feature_selected]
Xtest = X_test[['Age', 'Gender'] + feature_selected]

# Initialize and train Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(Xtrain, y_train)

# Predict probabilities for the positive class
y_pred_proba = rf.predict_proba(Xtest)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'{auc:.2f}')

0.72
